In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import random
import math
%matplotlib inline
from shapely.geometry import LineString
#For interactive

import geopandas as gpd
import contextily as cx

movShap = gpd.read_file(r"New/ShapeMatrice OD2016 - Passeggeri - Zone interne")
movShap_wm = movShap.to_crs(epsg=3857)

mov = pd.read_csv("New/matrice_od_2020_passeggeri/matrice_od_2020_passeggeri.csv")



In [ ]:
timeSlots = ['00:00-00:59', '01:00-01:59', '02:00-02:59', '03:00-03:59',
       '04:00-04:59', '05:00-05:59', '06:00-06:59', '07:00-07:59',
       '08:00-08:59', '09:00-09:59', '10:00-10:59', '11:00-11:59',
       '12:00-12:59', '13:00-13:59', '14:00-14:59', '15:00-15:59',
       '16:00-16:59', '17:00-17:59', '18:00-18:59', '19:00-19:59',
       '20:00-20:59', '21:00-21:59', '22:00-22:59', '23:00-23:59']
milanZones = ['','MILANO 1', 'MILANO 2', 'MILANO 3', 'MILANO 4', 'MILANO 5',
       'MILANO 6', 'MILANO 7', 'MILANO 8', 'MILANO 9', 'MILANO 10',
       'MILANO 11', 'MILANO 12', 'MILANO 13', 'MILANO 14', 'MILANO 15',
       'MILANO 16']
movTypes = ['COND', 'PAX', 'FERRO', 'GOMMA', 'MOTO', 'BICI', 'PIEDI', 'ALTRO']
movReason = ["STU","LAV","OCC","AFF","RIT"]

In [ ]:
movShap_wm_Mi = movShap_wm.loc[movShap_wm["desc_zona"].str.contains("MILANO")]#Can be changed

ax = movShap_wm_Mi.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
cx.add_basemap(ax, source="milanMap.tif")
plt.axis('off')
movShap_wm_Mi.apply(lambda x: ax.annotate(text=x['desc_zona'], xy=x.geometry.centroid.coords[0], ha='center',fontsize=20), axis=1);

## Store 1: Food in Milano 3

In [ ]:
ds1 = pd.read_excel("New/rdatibs4pertesisimonebellini/Food_B1_StoreFront_data.xlsx").dropna()
zonaStore = "MILANO 3"
currMov = mov.loc[mov["ZONA_DEST"] == zonaStore]            

In [ ]:
totMovTimeSlot = []
for ts in timeSlots:
    totMovTimeSlot.append(currMov.loc[currMov["FASCIA_ORARIA"] == ts].sum()[5:].sum()) #Total movements in that timeslot

#### Total movements

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(timeSlots,totMovTimeSlot)
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by time slot in " + zonaStore)
plt.show()

#### Where are movements from

In [ ]:
#Get where are from
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
whereFrom = []
for ts in timeSlots:
    movdf = currMov.loc[currMov["FASCIA_ORARIA"] == ts]#Get all movements for ts
    currGroup = movdf.groupby(['PROV_ORIG'], as_index=False).sum()
    whereFromTs = []
    for pog in currGroup.PROV_ORIG:#For each province
        whereFromTs.append(currGroup.loc[currGroup["PROV_ORIG"]== pog].sum()[1:].sum())#Get all movements from province and save
    whereFrom.append(whereFromTs)
labels = currGroup.PROV_ORIG



# Import Python cycling library
from itertools import cycle

# Create a colour code cycler e.g. 'C0', 'C1', etc.
color_codes = map('C{}'.format, cycle(range(len(whereFromTs))))

cm = plt.get_cmap('gist_rainbow')
ax.set_prop_cycle('color', [cm(1.*i/len(whereFromTs)) for i in range(len(whereFromTs))])


for i in range(0,len(whereFromTs)):
    if i ==0:
        plt.bar(timeSlots,list(list(zip(*whereFrom))[i]),label=labels.values[i])

        bot = np.array(list(list(zip(*whereFrom))[i]))
    else:
        plt.bar(timeSlots,list(list(zip(*whereFrom))[i]),label=labels.values[i],
               bottom=bot)
        bot += np.array(list(list(zip(*whereFrom))[i]))
plt.legend(loc="upper left",bbox_to_anchor=(1.2,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by origin in " + zonaStore)
plt.show()


In [ ]:
import plotly.express as px
origindf = pd.DataFrame(list(list(zip(*whereFrom))), columns =timeSlots).T
origindf.columns = labels
fig = px.bar(origindf, x=timeSlots,y = origindf.columns, title="Movements by origin in " + zonaStore)
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = origindf.index,
        ticktext = timeSlots)
)

fig.show()

In [ ]:
origindf.iloc[1].values

#### How people arrived there

In [ ]:
#Get how they arrived there
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
howArrive = []
for ts in timeSlots:
    movdf = currMov.loc[currMov["FASCIA_ORARIA"] == ts].sum()[5:].to_frame().T#Get all movements for ts
    howArrivets = []
    for typ in movTypes:#For each movement type (CAR, bike etc)
        filter_col = [col for col in aa if col.endswith(typ)]
        howArrivets.append(movdf[filter_col].sum().sum())
    howArrive.append(howArrivets)
    
for i in range(0,len(movTypes)):
    if i ==0:
        plt.bar(timeSlots,list(list(zip(*howArrive))[i]),label=movTypes[i])
        bot = np.array(list(list(zip(*howArrive))[i]))
    else:
        plt.bar(timeSlots,list(list(zip(*howArrive))[i]),label=movTypes[i],
               bottom=bot)
        bot += np.array(list(list(zip(*howArrive))[i]))
plt.legend(loc="upper left",bbox_to_anchor=(0.8,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by type in " + zonaStore)
plt.show()


#### Why people arrived there

In [ ]:
#Get why they arrived there
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
howArrive = []
for ts in timeSlots:
    movdf = currMov.loc[currMov["FASCIA_ORARIA"] == ts].sum()[5:].to_frame().T#Get all movements for ts
    howArrivets = []
    for typ in movReason:#For each movement type (CAR, bike etc)
        filter_col = [col for col in aa if col.startswith(typ)]
        howArrivets.append(movdf[filter_col].sum().sum())
    howArrive.append(howArrivets)
    
for i in range(0,len(movReason)):
    if i ==0:
        plt.bar(timeSlots,list(list(zip(*howArrive))[i]),label=movReason[i])
        bot = np.array(list(list(zip(*howArrive))[i]))
    else:
        plt.bar(timeSlots,list(list(zip(*howArrive))[i]),label=movReason[i],
               bottom=bot)
        bot += np.array(list(list(zip(*howArrive))[i]))
plt.legend(loc="upper left",bbox_to_anchor=(0.8,1.0))
ax.set_xticklabels(timeSlots,rotation='vertical')
ax.set_title("Movements by reason in " + zonaStore)
plt.show()
